In [ ]:
def mutateBORISandVID(depID,outloc,prh):

    # GET BORIS DATA
    borisloc = "C:\\01_Work\\1Project Home\\1Research\\GRAYS\\_tag data\\"+depID+"\\BORIS\\"       # path to folder with individual audit files
    borislist = os.listdir(borisloc)                                                               # list those files
    boris = pd.DataFrame()
    for f in borislist:
        vidname = pd.read_excel((borisloc+f), nrows = 1, header = None)
        data = pd.read_excel((borisloc+f), skiprows=15)
        data['vidname'] = vidname.iloc[0,1]
        boris = pd.concat([boris, data])
    #boris.to_excel(outloc+depID+"-tempcompiledBORIS.xlsx")

    # MUTATE BORIS DATA
    # first check the length of the audit file list against the length of vidDN to make sure they match since names may have changed
    if len(prh.get("vidDN")) != len(borislist):
        print('Differing number of audit files from prh vid DNs...')
    else:
        print('')

    dat = boris['vidname'].apply(lambda x: x.split('(')[1] if x.strip() else x) # remove the first (
    boris['vidnum'] = dat.apply(lambda x: x.split(')')[0]) # remove the secon ) and add to dataframe
    boris['vidnum'] = boris['vidnum'].str[:2] # take the first two numbers (for if a series of video numbers were combined)
    boris['vidnum'] = boris['vidnum'].apply(lambda x: x.lstrip('0') if x.strip() else x)
    # 02.12.2024. Changed to above to try to fix bug resulting if the video name is empty (happens when videos were merged during prh)
    #boris['vidnum']= boris['vidnum'].apply(lambda x: x.lstrip('0')) # remove the leading zeroes if present
    boris['vidnum'] = boris['vidnum'].astype(int)

    dat = prh.get('vidDN')
    dat = pd.DataFrame(dat.flatten())
    dat = dat.rename(columns = {0:'DN'})
    boris['vidDN'] = boris['vidnum'].apply(lambda x: dat['DN'].iloc[x] if x < len(dat) else None)

    # MUTATE VIDEO DATA
    viddat = pd.DataFrame()
    # pull the video name from prh file
    viddat['vidname'] = [item[0][0] for item in prh.get('vidNam')]
    # get the prh video number from the prh video name
    dat = viddat['vidname'].apply(lambda x: x.split('W')[1])
    viddat['vidnum'] = dat.apply(lambda x: x.split('.')[0])
    viddat['vidnum'].astype(int)
    viddat['vidnum']= viddat['vidnum'].apply(lambda x: x.lstrip('0'))
    # get the viddn
    viddat['vidDN'] = [item[0] for item in prh.get('vidDN')]
    # get the video start index
    dat = prh.get('DN')
    dat = dat.flatten()
    for i in range(len(viddat)): 
        searchDN = viddat.loc[viddat.index[i], 'vidDN']
        viddat.at[i,'indStartVid'] = np.where(dat >= searchDN)[0][0]
    
    # FINALIZE EDITS TO BORIS DATA
    # add the start index on the videos to the boris data
    viddat['vidnum'] = viddat['vidnum'].index
    boris = pd.merge(viddat, boris, how='inner', left_on='vidnum', right_on='vidnum')

    boris.drop(['Media file path', 'Total length', 'FPS'], axis = 1, inplace=True)
    boris.rename(columns={'vidname_x': 'prhvidname', 'vidname_y': 'borisvidname', 'vidDN_y':'vidDN'}, inplace=True)

    boris['ObsInd'] = round(boris['indStartVid']+(boris['Time']*fs))
    # get the observation ds
    for i, v in boris['vidDN'].items():
        boris.loc[i, 'vidDS'] = dt.fromordinal(int(v)) + timedelta(days=v%1) - timedelta(days = 366)
        boris.loc[i, 'obsDS'] = boris.loc[i, 'vidDS']+timedelta(seconds = boris.loc[i, 'Time'])

    boris.to_excel(outloc+depID+"-compiledBORIS.xlsx")
    print('Saved compiled BORIS file to directory...')
    return boris, viddat












